In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Introduction

Handle the null values in the dataset is one of the important steps in data wrangling. These null values adversely affect the performance and accuracy of any machine learning algorithm. So, it is very important to handle null values in the dataset before applying any machine learning algorithm to that dataset. Although some algorithms like XGBoost have built-in feature to handle null values, but we should also do it manually as a good practice while preparing the data.

# Removing Null Values from Dataset in Python using Pandas Library. 

In [ ]:
# We will use Titanic dataset
dataset = pd.read_csv('../input/titanicdataset-traincsv/train.csv')
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

we will just remove null values from our Titanic dataset as a part of data wrangling step in order to make our article short and crisp.

Lets see how many null values are there in our dataset?

In [ ]:
dataset.isnull().sum()

We will get a list of features with the count of null values. From the output of the above code, it is clear that Age column contains 177 null values and Cabin column contains 687 null values.

Lets drop the Cabin column

We see that Cabin column contains 687 null values out of 891 rows / observations. So, it makes sense to drop this column from the dataset. Lets drop it.

In [ ]:
# Please note that inplace parameter is used to permanently affect our dataset. 
# By default, it is false. If we don't set it to True explicitly, the Cabin column is not dropped permanently from our dataset.
dataset.drop('Cabin', axis=1, inplace=True)

In [ ]:
# Lets drop all the rows in the dataset which contain null values
# dataset.dropna(inplace=True)

It will remove all the rows which contain null values from the dataset. Now our dataset does not contain any null value. This step is not recommended. I added this step just for illustration. We can loose significant information by executing this step. There are methods to replace the null values with some meaningful values.

# Implement Imputer
Imputer class present in Scikit Learn library is used to replace the missing values in the numeric feature with some meaningful value like mean, median or mode. Lets see its implementation in Python using sklearn library.

In [ ]:
# Import the required libraries like pandas, numpy and sklearn
from sklearn.impute import SimpleImputer

In [ ]:
# I have added two features (added 10 values in each feature) in this data frame and deliberately put two nan values in the second feature. 
# We will impute these nan values using Imputer class present in sklearn library.

dataframe = pd.DataFrame()
dataframe['Feature_1'] = [0.42,0.56,0.36,0.90,0.98,0.64,0.76,0.56,0.39,0.77]
dataframe['Feature_2'] = [np.nan,0.90,0.75,0.45,np.nan,0.88,0.67,0.34,0.72,0.28]
dataframe

Explanation of Imputer class parameters:

missing_values — This is the value which has to be replaced in the dataset. This could either be an integer, or NaN. If you don’t pass this value, NaN will be the default value. So, wherever we have NaN in our dataset, the Imputer object will replace it with a new value.

strategy — This is the strategy we’ll be using to calculate the value which has to replace the NaN occurrences in the dataset. There are three different strategies we can use: mean, median, most_frequent.

verbose — This will just decide the verbosity of the Imputer. By default, it’s set to 0.

copy — This will decide if a copy of the original object has to be made, or if the Imputer should change the dataset in-place. By default, it is set to True.

In [ ]:
# Impute nan values with mean value using Imputer class

imputer = SimpleImputer(missing_values=np.nan, strategy='mean',verbose=0,copy="True")
dataframe = imputer.fit_transform(dataframe.values)
dataframe

# Replace missing values
We will use the Titanic dataset

In [ ]:
# Lets see how many missing values are there in the "Age" column of Titanic dataset
dataset['Age'].isnull().sum()

There are 177 missing values out of 891 observations. Now how to handle these 177 missing values? 

The general method to handle such kind of scenarios is to replace the missing values with some meaningful value. This meaningful values can be obtained by taking the mean, median or mode of all the not null values in the "Age" column. This is a statistical approach of handling the missing values and is well suited for linear data. 

In [ ]:
# Calculate mean, median and mode 

mean_age = dataset['Age'].mean()
median_age = dataset['Age'].median()
mode_age = dataset['Age'].mode()
display('Mean Age: ' + str(mean_age))
display('Median Age: ' + str(median_age))
display('Mode Age: ' + str(mode_age))

Replace the missing values in the "Age" column with any of the above calculated values. In this case, I am going to replace the missing values with the mean value.


In [ ]:
dataset['Age'].replace(np.NaN, mean_age)

Please note that, this is just an approximation of the missing values and it may lead to variance in the prediction but we have to deal with it. There is no way around. But this approach is far better than dropping the "Age" column due to which we will lose a lot of significant data.

# Replace missing value in Categorical Column

We had seen how to impute missing values in numeric columns, Now will see how can we impute missing values in categorical columns

Again, we will take example of Titanic dataset. There are two categorical columns (Cabin and Embarked) in the Titanic dataset which have missing values. Cabin has 687 missing values (out of 891), so its better to drop this column as it has more than 77% of null data. So, lets concentrate on Embarked column which has only 2 missing values.

In categorical columns, we introduce a new category usually called "Unknown" to impute missing values. As this column has 'S', 'C' and 'Q' categories, lets impute 'U' (Unknown) as a new category for 2 missing values.

In [ ]:
# We want to replace those nan values
dataset['Embarked'].unique()

In [ ]:
dataset['Embarked'].fillna('U',inplace=True)

In [ ]:
dataset['Embarked'].unique()

# Convert Categorical Variables into Dummies
Categorical variables are those variables which contain categorical values. For example, consider the "Sex" column in our Titanic dataset. It is categorical variable containing male and female. We need to convert these categories (male and female) into numbers (0 and 1) because most of the machine learning algorithms don't accept string values.

I found 3 categorical columns (Sex, Embarked and Pclass) in the Titanic dataset. So, lets convert them into numbers (0 and 1). Or, we can say lets one hot encode these variables.


Pandas library in Python contains get_dummies method which does the one hot encoding of the categorical variables (converts them into numbers - 0 and 1). The method get_dummies creates a new data frame which consists of zeros and ones. 

In [ ]:
# Convert categorical variables to their respective one hot encoded representation

sex = pd.get_dummies(dataset['Sex'],prefix="Sex")
embark = pd.get_dummies(dataset['Embarked'],prefix="Embarke")
pclass = pd.get_dummies(dataset['Pclass'],prefix="pclass")

In [ ]:
print(sex.head())
print(embark.head())
print(pclass.head())

In [ ]:
# Concatenate all the one hot encoded columns to the original dataset
dataset = pd.concat([dataset, sex, embark, pclass], axis=1)

In [ ]:
# Drop original columns
dataset.drop(['Sex', 'Embarked', 'Pclass'], axis=1, inplace=True)

In [ ]:
dataset

We can do the same thing using Label encoder and one hot encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [ ]:
# We will again import the Titanic dataset
data = pd.read_csv('../input/titanicdataset-traincsv/train.csv')
data

In [ ]:
# Filling the null values in embarked column with U as we've seen above
data['Embarked'].fillna('U',inplace=True)

In [ ]:
data['Sex'] = labelencoder.fit_transform(data['Sex'])
# data['Pclass'] = labelencoder.fir_transform(data['Sex']) # we won't do this as this column is already under integer values
data['Embarked'] = labelencoder.fit_transform(data['Embarked'])

In [ ]:
# Note that enbarked and sex columns have been changed into numeric column
data

But this transformation has its own problem. In column called Embarked which has 3 categorical values ('S', 'C', 'Q','U'). Label Encoder will convert these values into 0, 1, 2 and 3. Although in the original dataset, there is no relation between 'S', 'C', 'Q' and 'U'  but after label encoding it appears that there is some kind of relation like 'U' > 'Q' > 'C' > 'S' (which is not true) as 'U' is encoded to 3, 'Q' is encoded to 2, 'C' is encoded to 1 and 'S' is encoded to 0. So, in order to remove this confusion, we need to further use one hot encoding on it to create different columns corresponding to 'S', 'C' and 'Q' which will contain only zero and ones.


Convert the Label Encoded values to One Hot Encoded values

One Hot Encoder takes a column which has been label encoded, and then splits the column into multiple columns. The numbers are replaced by zeros and ones, depending on which column has what value.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [ ]:
sex = pd.DataFrame(enc.fit_transform(data[['Sex']]).toarray())
pclass = pd.DataFrame(enc.fit_transform(data[['Pclass']]).toarray())
embarked = pd.DataFrame(enc.fit_transform(data[['Embarked']]).toarray())

In [ ]:
print(sex)
print(pclass)
print(embarked)

In [ ]:
data = pd.concat([data,sex,pclass,embarked],axis=1)

In [ ]:
data.head()

Clearly not my favorite as you can see the mess above. Also, If any of you can suggest how can I add prefixes in the one hot encoding then please comment as I don't know it. 

# Remove invalid values from dataset in Python
Domain knowledge plays a crucial role in data wrangling. Sometimes, there are no missing values in the dataset but there are a lot of invalid values which we need to manually identify and remove those invalid values.

For example, consider "[Pima Indians Diabetes](https://www.kaggle.com/uciml/pima-indians-diabetes-database)" dataset which predicts the onset of diabetes within 5 years in Pima Indians, given medical details. This dataset has a lot of invalid values which we will try to remove in this article.

In [ ]:
df = pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
df.head()

In [ ]:
# Let's check null values
df.isnull().sum()

hmm, no null values that quit strange

In [ ]:
df.describe()

But there are minimum zero values in some columns. Here domain knowledge plays a vital role. Although there are no null or missing values in this dataset, but there are a lot of invalid values (zero) in the above column. Lets see how many zero values are there in the above columns:

In [ ]:
print((df[['Pregnancies','Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] == 0).sum())

In [ ]:
#In order to handle these invalid zero values, we will mark these values as NaN. 
df[['Pregnancies','Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = df[['Pregnancies','Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.NaN)

In [ ]:
# Now print null values:

df.isnull().sum()

So that's it for this notebook. Don't forget to upvote and comment your suggestions and feedback. Thanks!